In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler

# Cargar los datos
train_data = pd.read_csv('train.csv')

# Seleccionar las variables numéricas que están en el modelo
model_vars = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 
              'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 
              '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 
              'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
              '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

# Crear un subset con las variables del modelo
df_model = train_data[model_vars].copy()

# Remover filas con NaN
df_model.dropna(inplace=True)

# 1. Matriz de correlación
plt.figure(figsize=(20, 16))
correlation_matrix = df_model.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', 
            center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Matriz de Correlación entre Variables del Modelo', fontsize=16)
plt.tight_layout()


# 2. Correlación con SalePrice
plt.figure(figsize=(12, 10))
correlation_with_target = correlation_matrix['SalePrice'].sort_values(ascending=False)
sns.barplot(x=correlation_with_target.values[1:], y=correlation_with_target.index[1:])
plt.title('Correlación de Variables con SalePrice', fontsize=14)
plt.xlabel('Coeficiente de Correlación de Pearson')
plt.tight_layout()

# 3. Cálculo del VIF (Factor de Inflación de Varianza)
# Excluir SalePrice y crear una copia para el VIF
X = df_model.drop('SalePrice', axis=1)
X = add_constant(X)  # Añadir constante para statsmodels
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print("\nFactor de Inflación de Varianza (VIF):")
print(vif_data)

# 4. Visualizar VIF
plt.figure(figsize=(12, 10))
sns.barplot(x='VIF', y='Variable', data=vif_data[1:])  # Excluir la constante
plt.title('Factor de Inflación de Varianza por Variable', fontsize=14)
plt.axvline(x=10, color='r', linestyle='--', label='Umbral crítico (VIF=10)')
plt.axvline(x=5, color='orange', linestyle='--', label='Umbral de precaución (VIF=5)')
plt.legend()
plt.tight_layout()

# 5. Scatterplots de las principales variables vs SalePrice
top_correlated = correlation_with_target[1:6].index.tolist()
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, var in enumerate(top_correlated):
    sns.regplot(x=var, y='SalePrice', data=df_model, ax=axes[i], scatter_kws={'alpha':0.5})
    axes[i].set_title(f'{var} vs SalePrice', fontsize=12)
    
# Para el último subplot vacío
if len(top_correlated) < 6:
    axes[5].axis('off')
    
plt.tight_layout()

# 6. Análisis de Heterocedasticidad - Gráfico de Residuales
from sklearn.linear_model import LinearRegression

X = df_model.drop('SalePrice', axis=1)
y = df_model['SalePrice']

# Escalar variables para mejor interpretación
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ajustar modelo
model = LinearRegression()
model.fit(X_scaled, y)

# Predecir
y_pred = model.predict(X_scaled)

# Calcular residuales
residuals = y - y_pred

# Gráfico de residuales
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores Predictivos')
plt.ylabel('Residuales')
plt.title('Gráfico de Residuales del Modelo Completo')
plt.tight_layout()


# 7. Distribución de los residuales
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribución de los Residuales')
plt.xlabel('Residual')
plt.axvline(x=0, color='r', linestyle='--')
plt.tight_layout()


# 8. QQ-Plot para verificar normalidad de residuales
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot de Residuales')
plt.tight_layout()


# 9. Análisis de subgrupos de variables correlacionadas
# Superficie de sótano
basement_vars = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']
sns.pairplot(df_model[basement_vars + ['SalePrice']], height=2.5)
plt.suptitle('Relaciones entre Variables de Sótano', y=1.02, fontsize=16)
plt.tight_layout()

# Superficie habitable
living_vars = ['1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea']
sns.pairplot(df_model[living_vars + ['SalePrice']], height=2.5)
plt.suptitle('Relaciones entre Variables de Superficie Habitable', y=1.02, fontsize=16)
plt.tight_layout()


# Variables de garaje
garage_vars = ['GarageCars', 'GarageArea']
sns.pairplot(df_model[garage_vars + ['SalePrice']], height=2.5)
plt.suptitle('Relaciones entre Variables de Garaje', y=1.02, fontsize=16)
plt.tight_layout()


# Variables temporales
time_vars = ['YearBuilt', 'YearRemodAdd', 'YrSold']
sns.pairplot(df_model[time_vars + ['SalePrice']], height=2.5)
plt.suptitle('Relaciones entre Variables Temporales', y=1.02, fontsize=16)
plt.tight_layout()

print("Análisis de correlación y multicolinealidad completado. Revisa las gráficas generadas.")